#Subword Tokenization with the IMDB Reviews Dataset

##Download the IMDB reviews
- plain text
- subwords8k

In [ ]:
import tensorflow_datasets as tfds

#Downloading plain_text default config
imdb_plaintext, info_plaintext = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

#Doanloading subword encoded pretokenized dataset
imdb_subwords, info_subwords = tfds.load("imdb_reiews/subwords8k", with_info = )

##Subword Text Encoding

We first see the process with plaintext.

In [ ]:
# Get the train set
train_data = imdb_plaintext['train']
# Initialize sentences list
training_sentences = []
# Loop over all training examples and save to the list
for s,_ in train_data:
  training_sentences.append(s.numpy().decode('utf8'))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
oov_tok = '<OOV>'

#Initialize the Tokenizer class
tokenizer_plaintext = Tokenizer(num_words = 10000, oov_token = oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer_plaintext.fit_on_texts(training_sentences)

# Generate the training sequences
sequences = tokenizer_plaintext.texts_to_sequences(training_sentences)

- If we use this, there will be lots of OOV tokens when decoding using the lookup dictionary it created.




In [ ]:
sample_string = 'Tensorflow, from basics to mastery'

tokenized_string = tokenizer_plaintext.texts_to_sequences([sample_string])
original_string = tokenizer_plaintext.sequences_to_texts(tokenized_string)

print('Tokenized string is {}'.format(tokenized_string))
print('The original string is {}'.format(original_string))

Output:

``Tokenized string is [[1, 37, 1, 6, 1]]``

``The original string: ['<OOV> from <OOV> to <OOV>']``

- Increasing the `vocab_size` will just bloat the model.

**Subword text encoding** gets around this problem by using parts of the word to compose whole words. We can see the process of using subword text encoding by starting with the `encoder` object.

In [ ]:
tokenizer_subwords = info_subwords.features['text'].encoder

In [ ]:
tokenized_string = tokenizer_subwords.encode(sample_string)
original_string = tokenizer_subwords.decode(tokenized_string)

print ('Tokenized string is {}'.format(tokenized_string))
print ('The original string: {}'.format(original_string))

Output:

``Tokenized string is [6307, 2327, 4043, 2120, 2, 48, 4249, 4429, 7, 2652, 8050]``

``The original string: TensorFlow, from basics to mastery``

As you may notice, the sentence is correctly decoded. The downside is the token sequence is much longer. Instead of only 5 when using word-encoding, you ended up with 11 tokens instead. The mapping for this sentence is shown below:

In [ ]:
# Show token to subword mapping:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer_subwords.decode([ts])))

##Training the model

You will now train your model using this pre-tokenized dataset. Since these are already saved as sequences, you can jump straight to making uniform sized arrays for the train and test sets. These are also saved as `tf.data.Dataset` type so you can use the padded_batch() method to create batches and pad the arrays into a uniform size for training.

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64

# Get the train and test splits
train_data, test_data = imdb_subwords['train'], imdb_subwords['test'],

#Shuffle the training data
train_dataset = train_data.shuffle(BUFFER_SIZE)

#Batch and pad the datasets to the maximum length of the sequences
train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = train_data.padded_batch(BATCH_SIZE)

We then build the model.

In [ ]:
import tensorflow as tf

# Define dimensionality of the embedding
embedding_dim = 64

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer_subwords.vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

# Print the model summary
model.summary()

In [ ]:
num_epochs = 10

# Set the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Start training
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

##Visualizing the results

In [ ]:
import matplotlib.pyplot as plt

#plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

#plot the accuracy and results
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")